# 🧙🏻‍♂️ GANDALF

Gated Adaptive Network for Deep Automated Learning of Features (GANDALF): 
 - [Paper](https://arxiv.org/abs/2207.08548) 
 - [Model](https://pytorch-tabular.readthedocs.io/en/latest/models/#gated-adaptive-network-for-deep-automated-learning-of-features-gandalf)

# 📦 Setup

In [1]:
import datetime
import json
import os

import pandas as pd
import torch
import wandb
import numpy as np
from pytorch_tabular import TabularModel
from pytorch_tabular.config import (
    DataConfig,
    ExperimentConfig,
    OptimizerConfig,
    TrainerConfig,
)
from pytorch_tabular.models import (
    GANDALFConfig,
)
from rich import print
from rich.pretty import pprint

In [2]:
device = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
pprint(f"Using device: {device}")

'Using device: mps'

In [3]:
model = "GANDALF"
region_name = "promoters_512bp"
target = "MLL-N"
task = "multilabel_regression"
project = f"{model}_{task}_{region_name}_{target}"
start_time = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
sweep_name = f"sweep_multilabel_{model}_{region_name}_{target}_{start_time}"
run_name = f"run_{start_time}"
results_dir = f"results/{project}"
os.environ["WANDB_DIR"] = f"{results_dir}"
os.makedirs(results_dir, exist_ok=True)

pprint(f"Project: {project} | Sweep: {sweep_name}")

'Project: GANDALF_multilabel_regression_promoters_512bp_MLL-N | Sweep: sweep_multilabel_GANDALF_promoters_512bp_MLL-N_2025-04-12_1255'

# 📊 Load Data

In [4]:
data = pd.read_parquet(f"data/{region_name}.parquet")
for col in data.select_dtypes(include=["float64"]).columns:
    data[col] = data[col].astype("float32")
data = data.loc[:, ~data.columns.str.startswith("C3")]
data = data.loc[:, ~data.columns.str.startswith("C3")]
meth_cols = [col for col in data.columns if "METH" in col]
data[meth_cols] = data[meth_cols].fillna(-1)

train_data = data[~data.index.str.startswith(("chr8", "chr9"))]
val_data = data[data.index.str.startswith("chr8")]
test_data = data[data.index.str.startswith("chr9")]


# ⚙️ Config

In [5]:

optimizer_config = OptimizerConfig()

data_config = DataConfig(
    continuous_cols=[col for col in train_data.columns if target not in col],
    continuous_feature_transform="quantile_uniform",
    dataloader_kwargs={"persistent_workers": True},
    normalize_continuous_features=True,
    num_workers=10,
    pin_memory=True,
    target=[col for col in train_data.columns if target in col],
    validation_split=0,
)

def train():
    """Trains a model with the hyperparameters defined in the sweep."""
    if wandb.run is not None:
        wandb.finish()

    with wandb.init(
        name=run_name,
        reinit=True,
    ) as run:
        config = run.config

        trainer_config = TrainerConfig(
            accelerator="mps",
            auto_lr_find=True,
            batch_size=config.batch_size,
            check_val_every_n_epoch=5,
            checkpoints_path=f"{results_dir}/checkpoints",
            early_stopping_mode="min",
            early_stopping_patience=3,
            early_stopping="valid_loss",
            load_best=True,
            max_epochs=config.max_epochs,
            progress_bar="rich",
            trainer_kwargs=dict(enable_model_summary=False),
        )

        experiment_config = ExperimentConfig(
            exp_log_freq=5,
            exp_watch="gradients",
            log_logits=False,
            log_target="wandb",
            project_name=project,
            run_name=run.name,
        )

        model_config = GANDALFConfig(
            embedding_dropout=config.embedding_dropout,
            gflu_dropout=config.gflu_dropout,
            gflu_feature_init_sparsity=config.gflu_feature_init_sparsity,
            gflu_stages=config.gflu_stages,
            head="LinearHead",
            loss="MSELoss",
            metrics=["r2_score", "mean_squared_error"],
            seed=42,
            target_range = [(0, 1)] * len(data_config.target),
            task="regression",
        )

        model = TabularModel(
            data_config=data_config,
            experiment_config=experiment_config,
            model_config=model_config,
            optimizer_config=optimizer_config,
            trainer_config=trainer_config,
            verbose=False,
            suppress_lightning_logger=True,
        )

        model.fit(train=train_data, validation=val_data)
        model.predict(test_data)

# 🧹Sweep

In [ ]:
with open("config/sweep_config.json", "r") as f:
    sweep_config = json.load(f)

sweep_config["name"] = sweep_name
sweep_id = wandb.sweep(sweep_config, project=project)
wandb.agent(sweep_id=sweep_id, function=train, count=10, project=project)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: bt06mu84
Sweep URL: https://wandb.ai/catherine-chahrour-university-of-oxford/GANDALF_multilabel_regression_promoters_512bp_MLL-N/sweeps/bt06mu84


wandb: Agent Starting Run: mkoobhhu with config:
wandb: 	batch_size: 512
wandb: 	embedding_dropout: 0.10041147274844606
wandb: 	gflu_dropout: 0.03079751193862008
wandb: 	gflu_feature_init_sparsity: 0.4593175603371044
wandb: 	gflu_stages: 1
wandb: 	max_epochs: 88
wandb: Currently logged in as: catherine-chahrour (catherine-chahrour-university-of-oxford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


2025-04-12 12:55:18,774 - {pytorch_tabular.models.gandalf.gandalf:109} - INFO - Data Aware Initialization of T0

/Users/catherine/miniforge3/envs/stack/lib/python3.13/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


...

/Users/catherine/miniforge3/envs/stack/lib/python3.13/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/catherine/miniforge3/envs/stack/lib/python3.13/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/catherine/GMS/project/models/2025-04-11_gandalf/results/GANDALF_multilabel_regression_promoters_512bp_MLL-N/checkpoints exists and is not empty.


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2025-04-12 13:22:10,998 - {pytorch_tabular.models.gandalf.gandalf:109} - INFO - Data Aware Initialization of T0

wandb: WARNING Config item 'embedding_dropout' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gflu_stages' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gflu_dropout' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gflu_feature_init_sparsity' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_epochs' was locked by 'sweep' (ignored update).


Output()

# 🚂 Train

In [ ]:
pprint(sweep_id)

In [ ]:
# wandb.finish()